# <span style="color:orange"> Problems.<span>
*By Laura Lyons*
***

## <span style="color:orange"> Table of contents<span>
***
1. [Introduction](#1-introduction)
2. [Assigended Problems](#)\
   2.1. [Problem 01-Extending the Lady Tasting Tea](#problem-01---extending-the-lady-tasting-tea)\
   2.2. [Problem 02-Normal Distribution](#problem-02---normal-distribution)\
   2.3. [Problem 03-t-Tests](#problem-03----t-tests)\
   2.4. [Problem 04-ANOVA](#problem-04----anova)
3. [References](#references)


## <span style="color:orange"> 1. Introduction <span>

This notebook was created to fulfill an assessment requirement of Applied Statistics 25-25:(8651), Higher Diploma in Science, Data Analytics, as part of the H.Dip in Science in Data Analytics.

Each week, following a series of lectures, an problem was set, to demonstrate both learning and additional reading/research on the topics discussed in the lectures.

This notebook is a collection of the solutions to the problems set, including some additional guidance and instruction on how to run each assignment/program.

***

## <span style="color:orange">2. Assigned Problems<span>

## **Problem 01 - Extending the Lady Tasting Tea**

**1.1 Assignment Instructions:**

Let's extend the Lady Tasting Tea experiment as follows. The original experiment has 8 cups: 4 tea-first and 4 milk-first. Suppose we prepare 12 cups: 8 tea-first and 4 milk-first. A participant claims they can tell which was poured first.

Simulate this experiment using numpy by randomly shuffling the cups many times and calculating the probability of the participant correctly identifying all cups by chance. Compare your result with the original 8-cup experiment.

In your notebook, explain your simulation process clearly, report and interpret the estimated probability, and discuss whether, based on this probability, you would consider extending or relaxing the p-value threshold compared to the original design.

**1.2 Objective**

The objective of this problem is to simulate the *'Lady Testing Tea'* experiment and compare the probability of correctly identifyig all milk-first ups by changes in:
1. The original 8-cup setup (4 milk-first, 4 tea-first).
1. An extended 12-cup setup (4 milk-first, 8 tea-first).


First we need to import our prerequisites

In [10]:
# Mathematical functions from the standard library.
# https://docs.python.org/3/library/math.html
import math

# Permutations and combinations.
# https://docs.python.org/3/library/itertools.html
import itertools

# Random selections.
# https://docs.python.org/3/library/random.html
import random

# Numerical structures and operations.
# https://numpy.org/doc/stable/reference/index.html#reference
import numpy as np

# Plotting.
# https://matplotlib.org/stable/contents.html
import matplotlib.pyplot as plt

Initially we need to define some basic parameters, including the number of cups in total for each experiment,i.e. the original and the extended experient.

In [ ]:
# Number of cups of tea in total.
original_no_cups = 8
extended_no_cups = 12

# Number of cups of tea with milk in first.
original_no_cups_milk_first = 4
extended_no_cups_milk_first = 4

# Number of cups of tea with tea in first.
original_no_cups_tea_first = 4
extended_no_cups_tea_first = 8

# Number of ways of selecting four cups from eight.
original_ways = math.comb(original_no_cups, original_no_cups_milk_first)
extended_ways = math.comb(extended_no_cups, extended_no_cups_milk_first)

# Show original number of ways (answer= 70)
original_ways

70

In [ ]:
# Show extended number of ways (answer= 495)
extended_ways

495

In [ ]:
# Total number of ways of selecting four cups from eight, keeping them in order.
original_ways_ordered = 8 * 7 * 6 * 5

# Show total number of ordered ways (answer= 1680)
original_ways_ordered

1680

In [17]:
# Total number of ways of selecting four cups from 12, keeping them in order.
extended_ways_ordered = 12 * 11 * 10 * 9

# Show total number of ordered ways (answer= 11880)
extended_ways_ordered

11880

In this experiement, we need to build in the functionality to deal with the fact that:
- the order of the cup combination does not matter 
- and the same cup cannot be selected twice

In [ ]:
# No of ways of shuffling four cups.
number_shuffles = 4 * 3 * 2 * 1

# Number of combinations
original_number_of_combinations = original_ways_ordered // number_shuffles
# Floor division operator (//) was used to ensure that the answer retured is an integer, and not a float
# Reference: https://docs.python.org/3/tutorial/floatingpoint.html

# Show original number of combinations (answer= 24)
original_number_of_combinations

24

In [ ]:
# Number of combinations
extended_number_of_combinations = extended_ways_ordered // number_shuffles

# Show extended number of combinations (answer= 495)
extended_number_of_combinations

495

N

***

## **Problem 02 - Normal Distribution**

**Assignment Instructions:**

Generate 100,000 samples of size 10 from the standard normal distribution. For each sample, compute the standard deviation with ddof=1 (sample SD) and with ddof=0 (population SD). Plot histograms of both sets of values on the same axes with transparency. Describe the differences you see. Explain how you expect these differences to change if the sample size is increased.

***

## **Problem 03 -  t-Tests**

**Assignment Instructions:**

A type II error occurs when a test fails to reject the null hypothesis even though it is false.
For each mean difference $d = 0, 0.1, 0.2, \dots, 1.0$, repeat the following simulation 1,000 times:

1. Draw two samples of size 100, one from the standard normal distribution and one from the normal distribution with mean $d$ and standard deviation 1.
2. Run an independent samples t-test on the two samples, rejecting the null hypothesis if the p-value is less than 0.05.
3. Record the proportion of times the null hypothesis is not rejected.

Plot this proportion against $d$, and explain how the type II error rate changes as the difference in means increases.

***

## **Problem 04 -  ANOVA**

**Assignment Instructions:**

Generate three independent samples, each of size 30, from normal distributions with means 0, 0.5, and 1, each with standard deviation 1.

1. Perform a one-way ANOVA to test whether all three means are equal.
2. Perform three independent two-sample t-tests: samples 1 vs 2, 1 vs 3, and 2 vs 3.
3. Compare the conclusions.

Write a short note on why ANOVA is preferred over running several t-tests.

***

### End